# 搜索query相似计算

In [1]:
import pandas as pd
import tensorflow as tf
from gensim.models import Word2Vec
import jieba
from collections import Counter
from gensim.models import Doc2Vec
from gensim import corpora, models, similarities
import numpy as np
import time
import json

In [2]:
df = pd.read_json("./data/KUAKE-QQR_dev.json")
df

,id,query1,query2,label
0,s1,茴香是发物吗,茴香怎么吃？,0
1,s2,茴香是发物吗,茴香子怎么吃,0
2,s3,茴香是发物吗,茴香怎么吃最好,0
3,s4,茴香是发物吗,茴香的营养成分,0
4,s5,茴香是发物吗,茴香怎么吃,0
...,...,...,...,...
1595,s1596,月经来了十几天还没完是怎么回事,为什么月经来了十多天还未完,2
1596,s1597,月经来了十几天还没完是怎么回事,生理期10天还没完怎么回事,0
1597,s1598,月经来了十几天还没完是怎么回事,月经来了十天还没完,0
1598,s1599,月经来了十几天还没完是怎么回事,月经来了快二十天还没完是怎么回事,0


In [3]:
stop_words = [line.strip() for line in open("/Users/mac/Documents/我的坚果云/我的坚果云/learn_code/python_data/tech_data/jieba_dict/stop_words.txt").readlines()]

In [ ]:
query_01 = df["query1"].values

doc = [jieba.lcut(i) for i in query_01]

texts = [[word for word in doc[i] if word not in stop_words] for i in range(len(doc))]

In [ ]:
# 仅针对一个query
query_001 = df["query1"][0]
doc_001 = [jieba.lcut(query_001)]
text_001 = [[word for word in doc[i] if word not in stop_words] for i in range(len(doc))]

In [4]:
# 每一个query与每一个query进行相似度计算
result = []
for i in range(len(df)):
    query =""
    query = df["query1"][i]
    doc = [jieba.lcut(i) for i in (query)]
    text = [[word for word in doc[i] if word not in stop_words] for i in range(len(doc))]
    dic = corpora.Dictionary(text)
    corpus = [dic.doc2bow(text) for text in text]
    lsi = models.LsiModel(corpus, id2word=dic, power_iters = 100, num_topics=3)
    index = similarities.MatrixSimilarity(lsi[corpus])
    query_compare =df["query2"][i]
    vec_bow = dic.doc2bow(jieba.lcut(query_compare))
    vec_list = lsi[vec_bow]
    sims = index[vec_list]
    sims_avg = np.mean(sims)
    if  0.8 < sims_avg <= 1.0:
        label = 2
    elif 0.6 < sims_avg < 0.8:
        label = 1
    else:
        label = 0
    result.append([query,query_compare,sims_avg,label])
    # print("第"+str(i)+"个完成")
    time.sleep(3)
    i+=1

result
    

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/fr/n0ycpnx57f371zd5bf28dvj80000gp/T/jieba.cache
Loading model cost 0.830 seconds.
Prefix dict has been built successfully.


KeyboardInterrupt: 

In [ ]:
result_str = [str(i) for i in result].join(",")

file_name = "test.json"
with open(file_name,"w") as file_obj:
    json.dump(result_str,file_obj)

In [ ]:
# 尝试单独比一个，结果可以得出单条的相似度
dic_001 = corpora.Dictionary(text_001)
corpus_001 = [dic_001.doc2bow(text) for text in text_001]
lsi = models.LsiModel(
    corpus,
    id2word=dic_001,
    power_iters = 100,
    num_topics=3  # 主题
)
index_001 = similarities.MatrixSimilarity(lsi[corpus_001])

query_002 = df["query2"][0]

vec_bow = dic_001.doc2bow(jieba.lcut(query_002))
    # 计算query2 对应的向量
vec_lsi = lsi[vec_bow]
    # 计算query2 对应的向量与所有query1的相似度
sims = index_001[vec_lsi]
    # 输出原始query1，相似度 二元组
sims_avg = np.average(sims)
if  0.8 < sims_avg <= 1.0:
    label = 2
elif 0.6 < sims_avg < 0.8:
    label = 1
else:
    label = 0
result = [query_001, sims_avg,label]
result

In [ ]:
# 用gensim构建query词典，并记录词频
# 构建词典，给每个词编号
dic = corpora.Dictionary(texts)
# 每个query1里每个词出现频次
corpus = [dic.doc2bow(text) for text in texts]

In [ ]:
# 构建LSI模型
lsi = models.LsiModel(
    corpus,
    id2word=dic,
    power_iters = 100,
    num_topics=10  # 主题
)

In [ ]:
# lsi[corpus]是所有query1对应的向量
index_q = similarities.MatrixSimilarity(lsi[corpus])

In [ ]:

query_02 = df["query2"]

for item in query_02.iteritems():
    # 词袋模型，统计词频
    vec_bow = dic.doc2bow(jieba.lcut(item[1]))
    # 计算query2 对应的向量
    vec_lsi = lsi[vec_bow]
    # 计算query2 对应的向量与所有query1的相似度
    sims = index_q[vec_lsi]
    # 输出原始query1，相似度 二元组
    result =[(query_01[i[0]], i[1]) for i in enumerate(sims)]
result        